# RB175 Networked Applications

## Web server:
- The web server is responsible for using the operating system’s socket API to receive information in the form of HTTP requests.

## Application server:
- an HTTP interface that can translate HTTP requests, forwarded from the web server, into sensible arguments that are then passed to the application. This interface would also translate the application’s non-HTTP response into HTTP that is then passed back to the web server and, finally, to the client.
- It will execute your application just like an object calls another object’s method in Ruby.
- It will parse HTTP information into Ruby data structures that make sense.
- It will interface with a standard web server via socket connections.
- It will capture the return value of your application and transforms the return value into HTTP-formatted text.


The application server receives HTTP text from the web server, and then the application server’s HTTP parser will format this into Ruby-friendly data structures. When the application server is ready to hand the parsed request to the Ruby app, it will call your Ruby app with a method, passing in the parsed request as an argument or series of arguments. The Ruby app will execute, and the application server will store the Ruby app’s return value in a variable. It will re-format this variable into an HTTP response that the web server will understand, and then it will send that formatted HTTP response back to the web server via a socket connection. From there, the web server sends the HTTP response to the client.


## Rack
- How do we make sure that multuple combinations of different frameworks (application) can communicate with different application servers? 
- Rack is a generic interface to help application developers connect to applications servers.
- Rack’ is basically a specification of these two things: what the server should send to the app and what the app should return to the server. 

## Templating
- The ERB library can process a special syntax that mixes Ruby into HTML, and produces a final 100% HTML string.
- `<%= %>` will evaluate the embedded ruby code and include its return value in the HTML output (i.e. to call a method/variable).
- `<% %>` will only evaluate the Ruby code, but not include the return value in the HTML output (i.e. to define a method/variable).